## ASTR 21000, Winter 2020

## Laboratory on model parameter inference

### Distributed: Monday, March 2

import packages needed by the codes below. Run this cell first before using these codes. 

In [2]:
import numpy as np

# use jupyter "magic" command to tell it to embed plot into the notebook 
import matplotlib.pyplot as plt
%matplotlib inline

from codes.plot_utils import plot_pretty
plot_pretty(fontsize=12)

In [3]:
#zCMB, mB, emB were used before in hw 3
# x1 and ex1 are stretch parameter measured for each SN and its uncertainty
# csn and ecsn are color parameter and its uncertainty
zCMB, mB, emB, x1, ex1, csn, ecsn = np.loadtxt('data/jla_lcparams.txt', 
                                               usecols=(1, 4, 5, 6, 7, 8, 9), unpack=True)

print("read sample of %d supernovae..."%(np.size(zCMB)))

read sample of 740 supernovae...


In [6]:
#Set up parameters
from codes.cosmology import d_L_romberg
from scipy.interpolate import RectBivariateSpline
from codes.cosmology import d_L_vectorized
from codes.plot_utils import plot_color_map

clight = 2.99792458e5 # c in km/s
z, H0 = 2.0, 70.
Om0min, Om0max, OmLmin, OmLmax = 0., 1.,0.,1.
ntr = 10

def lum_grid(z, H0, Om0_vec, OmL_vec, atol=2.e-15, rtol=2.e-15):
    """
    helper function returns a grid of dL (scaled by H0/c)
    with n x n dimensions
    where n = the length of the Om0 and OmL vectors
    """
    n = np.size(Om0_vec)
    grid = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            grid[i][j] = (H0/clight)*d_L_vectorized(z, H0, Om0_vec[i], OmL_vec[j], atol=atol, rtol=rtol)
            #grid[i][j] = lum_scale(z, H0, Om0_vec[i],OmL_vec[j])
    return grid

def chebyshev_nodes1(a, b, N):
    return a + 0.5*(b-a)*(1. + np.cos((2.*np.arange(N+1)+1)*np.pi/(2.*(N+1))))

def polyfit2d(xtr, ytr, ftr, kx=3, ky=3, order=None):
    '''
    Returns polynomial spline coefficients for 2D polyfit w/ least squares
    
    Parameters:
    xtr, ytr: array-like, 1d
        xtr and ytr coordinates.
    ftr: 2d numpy array
        f(xgtr, ygtr) values evaluated on meshgrid of xtr and ytr vectors to fit by polynomial
    kx, ky: int, default is 3
        Polynomial order in x and y, respectively.
    order: int or None, default is None
    '''
    # grid coords
    x, y = np.meshgrid(xtr, ytr)
    # coefficient array, up to x^kx, y^ky
    coeffs = np.ones((kx+1, ky+1))

    # solve array
    V = np.zeros((coeffs.size, x.size))

    # construct Vandermonde matrix: for each coefficient produce array x^i, y^j
    for index, (j, i) in enumerate(np.ndindex(coeffs.shape)):
        # do not include powers greater than order
        if order is not None and i + j > order:
            arr = np.zeros_like(x)
        else:
            arr = coeffs[i, j] * x**i * y**j
        V[index] = arr.flatten()
        
    # do leastsq fitting and return leastsq result
    return np.linalg.lstsq(V.T, np.ravel(ftr), rcond=None)[0]

dLz = []
ntest = 100

Om0tr = chebyshev_nodes1(Om0min, Om0max, ntr-1)[::-1]
OmLtr = chebyshev_nodes1(OmLmin, OmLmax, ntr-1)[::-1]

Om0 = np.linspace(Om0min, Om0max, ntest)
OmL = np.linspace(OmLmin, OmLmax, ntest)

Om0_grid, OmL_grid = np.meshgrid(Om0, Om0, sparse=False, indexing='ij')

for iz, zi in enumerate(zCMB):
    #generate spline coefficients for dL approximation for each value of z
    px, py = 14,14
    dLtr_poly = lum_grid(zi, H0, Om0tr, OmLtr) #initalize training grid
    a = polyfit2d(Om0tr, OmLtr, dLtr_poly, kx=px, ky=py, order=None) #fit spline to data 
    dLz.append(a)

**Task 1a. (10 points)** Write a routine that computes $\ln L$ for this new 5-parameter model, where 

$$\ln L(\mathbf{y}\vert\mathbf{x}) = -\frac{1}{2}\,\sum\limits_{i=0}^{N_{\rm SN}-1}\frac{\Delta\mu^2}{\sigma_{\Delta\mu,i}^2}$$

and $\sigma_{\Delta\mu,i}^2$ is total uncertainty of the observational estimate of the distance modulus that accounts for uncertainties in $m_B$, $x_1$, and $c$, which by rules of error propagation is: 

$$\sigma_{\Delta\mu,i}^2 = \sigma_{m_B}^2 + \alpha^2\sigma_{x1}^2 + \beta^2\sigma_c^2,$$

which means that the error also depends on the two model parameters, $\alpha$ and $\beta$. 

In [9]:
from scipy.optimize import differential_evolution as de
def fit_dl(Om0_vec, OmL_vec, a):
    Om0_grid, OmL_grid = np.meshgrid(Om0_vec, OmL_vec, sparse=False, indexing='ij')
    fitted = np.polynomial.polynomial.polyval2d(Om0_grid, OmL_grid, a.reshape((px+1,py+1)))
    return fitted
def logL2(x):
    """
    Parameters:
    x: vector containing Om0, OmL, tM0, alpha, beta
    
    Returns:
    -2 ln(L)
    """
    Omega0, OmegaL, tildeM0, a, b = x
    logLZ = 0

    for iz, zi in enumerate(zCMB):
        dL = fit_dl(Omega0, OmegaL, dLz[iz])
        dMu = mB[iz]-5*np.log10(dL)-tildeM0+a*x1[iz]-b*csn[iz]
        
        sigma2 = emB[iz]**2 + (a*ex1[iz])**2 + (b*ecsn[iz])**2
        lTemp = (dMu**2)/sigma2
        logLZ = logLZ + lTemp
    return logLZ

**Task 1b. (8 points)** 
The parameters that minimize $-2\ln L$ are: $\Omega_{\rm m0} \approx 0.252, \Omega_\Lambda \approx 0.703, \tilde{M}_0 \approx 24.075, \alpha = 0.130, \beta = 3.174$. These parameters differ from the ones that minimize the $\chi^2$ value in Homework 3: $\Omega_{\rm m0} \approx 0.468, \Omega_\Lambda \approx 0.516, \tilde{M}_0 \approx 24.089$. 
While the $\tilde{M}_0$ value is similar using both minimization methods, the $\Omega_{\rm m0}$ estimated using the 5-parameter model is smaller and the $\Omega_\Lambda$ is larger. This 5-parameter model is more accurate, and the estimated $\Omega_\Lambda = 0.703$ that minimizes error associated with the likelihood function is very close to the accepted value of $\Omega_\Lambda$ around 0.68.

In [21]:
#define bounds for minimization
bounds = [(0,1.),(0,1.),(20.,28.),(0.05,0.3),(1.,5.)]
print(result.x, result.fun)

5
[ 0.25203076  0.70298803 24.07510902  0.12999434  3.17389316] [[622.71339169]]


**Task 1c (2 points).

The reduced $\chi^2$ for the values of the parameters that minimize the $-2\ln L$ is $\approx 3.68$. This suggests that the 5-parameter model describes the supernova measurements more accurately than the 3-parameter value, because this reduced $\chi^2$ is smaller than the minimum reduced $\chi^2$ obtained from Homework 3, which was $\approx 6.21$.

In [26]:
def redChi2(params):
    Omega0, OmegaL, tildeM0, a, b = params
    
    chiZ = np.empty(np.size(zCMB))
    for i, zi in enumerate(zCMB):
        #dL approx corresponding to the ith supernova
        dL_approx = fit_dl(Omega0, OmegaL, dLz[i]) #2d grid for supernova
        dMu = mB[i] - 5*np.log10(dL_approx) - tildeM0 +a*x1[iz]-b*csn[iz]
        sigma2 = emB[iz]**2 + (a*ex1[iz])**2 + (b*ecsn[iz])**2
        chiZ[i] = (dMu**2)/sigma2
    
    #return reduced chi2 value
    return np.sum(chiZ)/(len(zCMB) - len(params))

redChi2(result.x)

3.6810282290021292

#### Group work

If you feel that your group work in hw 3 was productive, I encourage you to continue working with your group. You can also form a new group, if you wish. In this type of exercise it is helpful to discuss approach and results in a group and such collaborative work is encouraged. 